<a href="https://colab.research.google.com/github/luisosmx/Python_exercises/blob/main/corelogicapi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import requests
import pandas as pd


class PropertySearch:
    def __init__(self, base_url, client_credentials_url, client_id, client_secret):
        self.base_url = base_url
        self.client_credentials_url = client_credentials_url
        self.client_id = client_id
        self.client_secret = client_secret
        self.token = None

    def authenticate(self):
        auth_data = "grant_type=client_credentials"

        auth_response = requests.post(
            self.client_credentials_url + "?" + auth_data,
            auth=(self.client_id, self.client_secret),
            # data=auth_data
        )

        if auth_response.status_code == 200:
            auth_response_data = auth_response.json()
            self.token = auth_response_data.get("access_token")
            print(f"Authentication successful.")
        else:
            print(f"Authentication failed. Status Code: {auth_response.status_code}")

    def search_property(self, street_address, city, state, zip_code):
        if not self.token:
            print("Please authenticate first.")
            return None

        self.headers = {
                "Authorization": f"Bearer {self.token}",
                "Content-Type": "application/json"
            }

        params = {
            "streetAddress": street_address,
            "city": city,
            "state": state,
            "zipCode": zip_code
        }

        self.response = requests.get(self.base_url, headers=self.headers, params=params)

        if self.response.status_code == 200:
            return self.response.json()
        else:
            print(f"Error: {self.response.status_code}")
            return None

    def property_detail(self, response):

        self.clip_value = response['items'][0]['clip']
        url = f'https://property.corelogicapi.com/v2/properties/{self.clip_value}/property-detail'
        self.resource_detail = requests.get(url, headers= self.headers)

        if self.resource_detail.status_code == 200:
              return self.resource_detail.json()
        else:
            print(f"Error: {self.resource_detail.status_code}")
            return None

    def extract_relevant_data(self, response, resource_detail):
        try:
            building_data = {
                'clip': response['items'][0]['clip'],
                'yearBuilt': resource_detail['buildings']['data']['buildings'][0]['constructionDetails']['yearBuilt'],
                'streetAddress': response['items'][0]['propertyAddress']['streetAddress'],
                'city': response['items'][0]['propertyAddress']['city'],
                'state': response['items'][0]['propertyAddress']['state'],
                'zipCode': response['items'][0]['propertyAddress']['zipCode'],
                'propertyMatchScore': response['items'][0]['addressMatchInformation']['propertyMatchScore'],
                'Tax_Year': resource_detail['taxAssessment']['items'][0]['assessedValue']['taxAssessedYear'],
                'Total_Tax_Amount': resource_detail['taxAssessment']['items'][0]['taxAmount']['totalTaxAmount'],
                'Land_Value': resource_detail['taxAssessment']['items'][0]['assessedValue']['calculatedLandValue'],
                'Total_Value': resource_detail['taxAssessment']['items'][0]['assessedValue']['calculatedTotalValue'],
                'Improvement_Value': resource_detail['taxAssessment']['items'][0]['assessedValue']['calculatedImprovementValue']
            }

            building_df = pd.DataFrame([building_data])

            return building_df

        except Exception as e:
            print(f"An error occurred: {str(e)}")
            return None

client_id = ''
client_secret = ''
property_search_url = "https://property.corelogicapi.com/v2/properties/search"
auth_url = "https://api-prod.corelogic.com/oauth/token"

try:
    property_search = PropertySearch(property_search_url, auth_url, client_id, client_secret)

    # Authenticate to obtain the access token
    property_search.authenticate()

    # Now you can use the search_property method
    result = property_search.search_property("31899 Highway 41", "Coarsegold", "CA", "93614")

    if result:
        print("Property Search Result:")
        print(result)

        resource_url = property_search.property_detail(result)
        print('Property Detail:')
        print(resource_url)

        result_df = property_search.extract_relevant_data(result, resource_url)
        print(result_df)
except Exception as e:
    print(f"An error occurred: {str(e)}")


In [ ]:
 # Generate a method that creates a request to property detail endpoint
    # Extract clip id from result search_property
    # Create the request
    # Analize the response and identify the foelds that give property value information
    # Create a dataframe that contains
        # Name of the property
        # Address
        # etc
        # property value

In [ ]:
{
    "buildings": {
        "data": {
            "clip": "2562760795",
            "allBuildingsSummary": {
                "buildingsCount": 1,
                "unitsCount": null,
                "roomsCount": null,
                "bedroomsCount": null,
                "bathroomsCount": 1,
                "fullBathroomsCount": 1,
                "halfBathroomsCount": null,
                "oneQtrBathroomsCount": null,
                "threeQtrBathroomsCount": null,
                "bathroomFixturesCount": null,
                "fireplacesCount": null,
                "livingAreaSquareFeet": 768,
                "totalAreaSquareFeet": 768,
                "openAreasSquareFeet": null,
                "officeSpaceSquareFeet": null,
                "elevatorsCount": null,
                "loadingDocksCount": null,
                "railSpursCount": null,
                "truckDoorsCount": null
            },
            "buildings": [
                {
                    "structureId": {
                        "sequenceNumber": 1,
                        "compositeBuildingLinkageKey": "06039054330002                                    001001",
                        "buildingName": null,
                        "buildingNumber": "1",
                        "buildingSectionNumber": null,
                        "buildingComments": null
                    },
                    "structureClassification": {
                        "buildingTypeCode": null,
                        "buildingClassCode": null,
                        "gradeTypeCode": null,
                        "fireSprinklerTypeCode": null,
                        "fireInsuranceTypeCode": null
                    },
                    "structureFootprint": {
                        "widthFeet": null,
                        "depthFeet": null
                    },
                    "structureUnitsSummary": {
                        "vacantCount": null,
                        "residentialCount": null,
                        "commercialCount": null
                    },
                    "structureVerticalProfile": {
                        "storiesCount": null,
                        "storiesTypeCode": null,
                        "floorNumber": null
                    },
                    "constructionDetails": {
                        "yearBuilt": 1920,
                        "effectiveYearBuilt": 1920,
                        "buildingStyleTypeCode": null,
                        "buildingQualityTypeCode": null,
                        "frameTypeCode": null,
                        "foundationTypeCode": null,
                        "constructionTypeCode": null,
                        "buildingRemodelTypeCode": null,
                        "buildingImprovementConditionCode": null,
                        "buildingImprovementTypeCode": null,
                        "buildingImprovementValue": null
                    },
                    "structureExterior": {
                        "patios": {
                            "count": null,
                            "typeCode": null,
                            "areaSquareFeet": null
                        },
                        "porches": {
                            "count": null,
                            "typeCode": null,
                            "areaSquareFeet": null,
                            "secondPorchAreaSquareFeet": null
                        },
                        "parking": {
                            "typeCode": null,
                            "garageTypeCode": null,
                            "parkingSpacesCount": null,
                            "primaryAreaSquareFeet": null,
                            "secondAreaSquareFeet": null,
                            "carportAreaSquareFeet": null
                        },
                        "pool": null,
                        "walls": {
                            "typeCode": null
                        },
                        "roof": {
                            "typeCode": null,
                            "coverTypeCode": null
                        }
                    },
                    "structureInterior": {
                        "attic": {
                            "typeCode": null
                        },
                        "walls": {
                            "typeCode": null
                        },
                        "basement": {
                            "typeCode": null,
                            "finishTypeCode": null,
                            "finishPercent": null
                        },
                        "flooring": {
                            "typeCode": null,
                            "coverTypeCode": null
                        },
                        "ceiling": {
                            "typeCode": null,
                            "heightFeet": null
                        },
                        "bathroomFixtures": {
                            "count": null
                        }
                    },
                    "interiorArea": {
                        "universalBuildingAreaSquareFeet": 768,
                        "universalBuildingAreaSquareFeetSourceCode": "B",
                        "buildingAreaSquareFeet": 768,
                        "buildingAdjustedAreaSquareFeet": null,
                        "buildingGrossAreaSquareFeet": null,
                        "livingAreaSquareFeet": 768,
                        "aboveGradeAreaSquareFeet": null,
                        "groundFloorAreaSquareFeet": null,
                        "basementAreaSquareFeet": null,
                        "finishedBasementAreaSquareFeet": null,
                        "unfinishedBasementAreaSquareFeet": null,
                        "aboveGroundFloorAreaSquareFeet": null,
                        "buildingAdditionsAreaSquareFeet": null,
                        "entryLevelFloorAreaSquareFeet": null,
                        "secondFloorAreaSquareFeet": null,
                        "thirdFloorAreaSquareFeet": null
                    },
                    "interiorRooms": {
                        "totalCount": null,
                        "bedroomsCount": null,
                        "bathroomsCount": null,
                        "fullBathroomsCount": 1,
                        "halfBathroomsCount": null,
                        "oneQtrBathroomsCount": null,
                        "threeQtrBathroomsCount": null,
                        "kitchensCount": null,
                        "familyRoomsCount": null,
                        "livingRoomsCount": null,
                        "basementRoomsCount": null
                    },
                    "structureFeatures": {
                        "airConditioning": {
                            "typeCode": null
                        },
                        "firePlaces": {
                            "typeCode": null,
                            "count": null
                        },
                        "heating": {
                            "typeCode": null
                        },
                        "plumbing": {
                            "typeCode": null
                        },
                        "passengerElevators": {
                            "count": null
                        },
                        "dormerWindows": {
                            "count": null
                        }
                    }
                },
                {
                    "structureId": {
                        "sequenceNumber": 2,
                        "compositeBuildingLinkageKey": "06039054330002                                    001002",
                        "buildingName": null,
                        "buildingNumber": "2",
                        "buildingSectionNumber": 1,
                        "buildingComments": null
                    },
                    "structureClassification": {
                        "buildingTypeCode": null,
                        "buildingClassCode": null,
                        "gradeTypeCode": null,
                        "fireSprinklerTypeCode": null,
                        "fireInsuranceTypeCode": null
                    },
                    "structureFootprint": {
                        "widthFeet": null,
                        "depthFeet": null
                    },
                    "structureUnitsSummary": {
                        "vacantCount": null,
                        "residentialCount": null,
                        "commercialCount": null
                    },
                    "structureVerticalProfile": {
                        "storiesCount": null,
                        "storiesTypeCode": null,
                        "floorNumber": null
                    },
                    "constructionDetails": {
                        "yearBuilt": 1920,
                        "effectiveYearBuilt": 1920,
                        "buildingStyleTypeCode": "LSH",
                        "buildingQualityTypeCode": null,
                        "frameTypeCode": null,
                        "foundationTypeCode": null,
                        "constructionTypeCode": "FRM",
                        "buildingRemodelTypeCode": null,
                        "buildingImprovementConditionCode": null,
                        "buildingImprovementTypeCode": null,
                        "buildingImprovementValue": null
                    },
                    "structureExterior": {
                        "patios": {
                            "count": null,
                            "typeCode": null,
                            "areaSquareFeet": null
                        },
                        "porches": {
                            "count": null,
                            "typeCode": null,
                            "areaSquareFeet": null,
                            "secondPorchAreaSquareFeet": null
                        },
                        "parking": {
                            "typeCode": null,
                            "garageTypeCode": null,
                            "parkingSpacesCount": null,
                            "primaryAreaSquareFeet": null,
                            "secondAreaSquareFeet": null,
                            "carportAreaSquareFeet": null
                        },
                        "pool": null,
                        "walls": {
                            "typeCode": null
                        },
                        "roof": {
                            "typeCode": null,
                            "coverTypeCode": null
                        }
                    },
                    "structureInterior": {
                        "attic": {
                            "typeCode": null
                        },
                        "walls": {
                            "typeCode": null
                        },
                        "basement": {
                            "typeCode": null,
                            "finishTypeCode": null,
                            "finishPercent": null
                        },
                        "flooring": {
                            "typeCode": null,
                            "coverTypeCode": null
                        },
                        "ceiling": {
                            "typeCode": null,
                            "heightFeet": null
                        },
                        "bathroomFixtures": {
                            "count": null
                        }
                    },
                    "interiorArea": {
                        "universalBuildingAreaSquareFeet": null,
                        "universalBuildingAreaSquareFeetSourceCode": null,
                        "buildingAreaSquareFeet": null,
                        "buildingAdjustedAreaSquareFeet": null,
                        "buildingGrossAreaSquareFeet": null,
                        "livingAreaSquareFeet": null,
                        "aboveGradeAreaSquareFeet": null,
                        "groundFloorAreaSquareFeet": null,
                        "basementAreaSquareFeet": null,
                        "finishedBasementAreaSquareFeet": null,
                        "unfinishedBasementAreaSquareFeet": null,
                        "aboveGroundFloorAreaSquareFeet": null,
                        "buildingAdditionsAreaSquareFeet": null,
                        "entryLevelFloorAreaSquareFeet": null,
                        "secondFloorAreaSquareFeet": null,
                        "thirdFloorAreaSquareFeet": null
                    },
                    "interiorRooms": {
                        "totalCount": null,
                        "bedroomsCount": null,
                        "bathroomsCount": null,
                        "fullBathroomsCount": 1,
                        "halfBathroomsCount": null,
                        "oneQtrBathroomsCount": null,
                        "threeQtrBathroomsCount": null,
                        "kitchensCount": null,
                        "familyRoomsCount": null,
                        "livingRoomsCount": null,
                        "basementRoomsCount": null
                    },
                    "structureFeatures": {
                        "airConditioning": {
                            "typeCode": null
                        },
                        "firePlaces": {
                            "typeCode": null,
                            "count": null
                        },
                        "heating": {
                            "typeCode": null
                        },
                        "plumbing": {
                            "typeCode": null
                        },
                        "passengerElevators": {
                            "count": null
                        },
                        "dormerWindows": {
                            "count": null
                        }
                    }
                }
            ]
        }
    },
    "ownership": {
        "data": {
            "clip": "2562760795",
            "currentOwners": {
                "ownerNames": [
                    {
                        "sequenceNumber": 1,
                        "fullName": "COARSEGOLD HISTORICAL SOCIETY INC",
                        "firstNameAndMiddleInitial": null,
                        "lastName": null,
                        "firstName": null,
                        "middleName": null,
                        "isCorporate": true
                    },
                    {
                        "sequenceNumber": 2,
                        "fullName": null,
                        "firstNameAndMiddleInitial": null,
                        "lastName": null,
                        "firstName": null,
                        "middleName": null,
                        "isCorporate": false
                    },
                    {
                        "sequenceNumber": 3,
                        "fullName": null,
                        "firstNameAndMiddleInitial": null,
                        "lastName": null,
                        "firstName": null,
                        "middleName": null,
                        "isCorporate": false
                    },
                    {
                        "sequenceNumber": 4,
                        "fullName": null,
                        "firstNameAndMiddleInitial": null,
                        "lastName": null,
                        "firstName": null,
                        "middleName": null,
                        "isCorporate": false
                    }
                ],
                "relationshipTypeCode": null,
                "ownerEtalCode": null,
                "occupancyCode": null,
                "ownershipRightsCode": "CO"
            },
            "currentOwnerMailingInfo": {
                "mailingAddress": {
                    "careOfName": null,
                    "streetAddress": "PO BOX 117",
                    "streetAddressParsed": {
                        "houseNumber": null,
                        "houseNumberSuffix": null,
                        "houseNumber2": null,
                        "direction": null,
                        "streetName": "PO BOX 117",
                        "mailingMode": null,
                        "quadrant": null,
                        "unitNumber": null
                    },
                    "city": "COARSEGOLD",
                    "state": "CA",
                    "zipCode": "93614",
                    "carrierRoute": "B001",
                    "foreignAddress": null
                },
                "ownerMailingOptOutIndicator": null
            }
        }
    },
    "siteLocation": {
        "data": {
            "clip": "2562760795",
            "coordinatesParcel": {
                "lat": 37.212065,
                "lng": -119.707243
            },
            "coordinatesBlock": {
                "lat": 37.212411,
                "lng": -119.708888
            },
            "locationLegal": {
                "subdivisionName": null,
                "subdivisionTractNumber": null,
                "subdivisionPlatBookNumber": null,
                "subdivisionPlatPageNumber": null,
                "blockNumber": null,
                "blockNumberSuffix": null,
                "lotNumber": null,
                "lotNumberSuffix": null,
                "description": "SEC 29 T8S R21E"
            },
            "locationSurvey": {
                "range": "21E",
                "township": "08S",
                "section": "29",
                "quarterSection": null
            },
            "neighborhood": {
                "code": null,
                "name": null
            },
            "municipality": {
                "code": null,
                "name": null
            },
            "town": {
                "code": null
            },
            "jurisdictionCounty": {
                "code": null
            },
            "cbsa": {
                "code": "31460",
                "type": "Metro"
            },
            "censusTract": {
                "id": "0001102028"
            },
            "taxRateArea": {
                "code": "062001"
            },
            "taxDistrict": {
                "name": null
            },
            "landUseAndZoningCodes": {
                "propertyTypeCode": "00",
                "landUseCode": "999",
                "stateLandUseCode": null,
                "stateLandUseDescription": null,
                "countyLandUseCode": "4PXX",
                "countyLandUseDescription": "MISC STRUCT INSTIT",
                "zoningCode": "IA",
                "zoningCodeDescription": "INSTITUTION AREA DISTRICT",
                "isManufacturedHome": null
            },
            "lot": {
                "areaAcres": 1.97,
                "areaSquareFeet": 85813,
                "areaSquareFeetUsable": null,
                "depthFeet": null,
                "frontFeet": null,
                "shapeCode": null,
                "topographyType": null,
                "easementTypeCode": null
            },
            "utilities": {
                "fuelTypeCode": null,
                "electricityWiringTypeCode": null,
                "sewerTypeCode": null,
                "utilitiesTypeCode": null,
                "waterTypeCode": null
            }
        }
    },
    "taxAssessment": {
        "metadata": {
            "pageNumber": 1,
            "pageSize": 1,
            "totalRecords": 1,
            "totalPages": 1
        },
        "items": [
            {
                "clip": "2562760795",
                "taxAmount": {
                    "billedYear": null,
                    "delinquentYear": null,
                    "areaCode": "062001",
                    "areaCodeDescription": null,
                    "propertyTaxRate": null,
                    "calculatedTotalExemptionAmount": 72359,
                    "totalTaxExemptionAmount": null,
                    "totalTaxAmount": null,
                    "countyTaxAmount": null,
                    "schoolTaxAmount": null,
                    "townTaxAmount": null,
                    "villageTaxAmount": null,
                    "netTaxAmount": null
                },
                "taxExemptions": {
                    "commercial": [
                        "CHARITY"
                    ],
                    "residential": []
                },
                "assessedValue": {
                    "taxAssessedYear": 2022,
                    "calculatedTotalValue": 72359,
                    "calculatedLandValue": 66638,
                    "calculatedImprovementValue": 5721,
                    "calculatedImprovementValuePercentage": 8,
                    "calculatedTotalValueSourceCode": "A",
                    "taxableValue": null,
                    "taxableImprovementValue": null,
                    "taxableLandValue": null,
                    "taxableOtherValue": null
                },
                "taxrollUpdate": {
                    "lastAssessorUpdateDate": "2023-01-11",
                    "taxrollCertificationDate": "2022-07-01"
                },
                "schoolDistricts": {
                    "school": {
                        "code": "YOSEMITE",
                        "name": "YOSEMITE",
                        "elementary": {
                            "code": "COARSEGOLD UN",
                            "name": null
                        },
                        "middle": {
                            "code": null,
                            "name": null
                        },
                        "high": {
                            "code": null,
                            "name": null
                        },
                        "communityCollege": {
                            "code": "STATE CENTER",
                            "name": null
                        }
                    }
                },
                "serviceDistricts": {
                    "fire": {
                        "code": null,
                        "name": null
                    },
                    "trash": {
                        "code": null,
                        "name": null
                    },
                    "lighting": {
                        "code": null,
                        "name": null
                    },
                    "tax": {
                        "code": null,
                        "name": null
                    },
                    "sewer": {
                        "code": null,
                        "name": null
                    },
                    "utility": {
                        "code": null,
                        "name": null
                    },
                    "water": {
                        "code": null,
                        "name": null
                    }
                }
            }
        ]
    },
    "mostRecentOwnerTransfer": {
        "metadata": {
            "pageNumber": 1,
            "pageSize": 1,
            "totalRecords": 1,
            "totalPages": 1
        },
        "items": [
            {
                "clip": "2562760795",
                "transactionDetails": {
                    "primaryCategoryCode": "A",
                    "deedCategoryCode": "G",
                    "saleDateDerived": "1992-04-13",
                    "saleRecordingDateDerived": "1992-04-13",
                    "saleAmount": 40000,
                    "saleTypeCode": "F",
                    "saleDocumentTypeCode": "ID",
                    "saleDocumentNumber": "8893",
                    "saleBookNumber": "",
                    "salePageNumber": "",
                    "ownershipTransferPercent": null,
                    "multiOrSplitParcelCode": "",
                    "isCashPurchase": false,
                    "isMortgagePurchase": true,
                    "isInterfamilyRelated": true,
                    "isInvestorPurchase": false,
                    "isResale": true,
                    "isShortSale": false,
                    "isForeclosureReo": false,
                    "isForeclosureReoSale": false
                },
                "recordedPropertyAddress": {
                    "streetAddress": "",
                    "streetAddressParsed": {
                        "houseNumber": "31899",
                        "houseNumberSuffix": "",
                        "houseNumber2": "",
                        "direction": "",
                        "streetName": "HIGHWAY 41",
                        "mode": "",
                        "quadrant": "",
                        "unitNumber": ""
                    },
                    "city": "COARSEGOLD",
                    "state": "CA",
                    "zipCode": "936149742",
                    "carrierRoute": "H069",
                    "county": "MADERA"
                },
                "titleCompany": {
                    "name": "",
                    "code": "00006"
                },
                "propertyDetails": {
                    "actualYearBuilt": null,
                    "effectiveYearBuilt": null,
                    "isResidentialProperty": null,
                    "isNewConstruction": false
                },
                "landUseAndZoningCodes": {
                    "propertyTypeCode": "",
                    "landUseCode": "",
                    "stateLandUseDescription": null,
                    "countyLandUseDescription": null,
                    "zoningCode": ""
                },
                "buyerDetails": {
                    "buyerNames": [],
                    "relationshipTypeCode": "",
                    "etalCode": "",
                    "occupancyCode": "",
                    "ownershipRightsCode": "CO",
                    "mailingAddress": {
                        "careOfName": "",
                        "streetAddress": "PO BOX 117",
                        "streetAddressParsed": {
                            "houseNumber": "",
                            "houseNumberSuffix": "",
                            "houseNumber2": "",
                            "direction": "",
                            "streetName": "PO BOX    117",
                            "mode": "",
                            "quadrant": "",
                            "unitNumber": ""
                        },
                        "city": "",
                        "state": "CA",
                        "zipCode": "936140117",
                        "carrierRoute": "B001"
                    },
                    "hasPartialInterest": false,
                    "mailingOptOutIndicator": null
                },
                "sellerDetails": {
                    "sellerNames": [
                        {
                            "sequenceNumber": 1,
                            "fullName": "KERNS TERI"
                        }
                    ]
                }
            }
        ]
    },
    "lastMarketSale": {
        "metadata": {
            "pageNumber": 1,
            "pageSize": 1,
            "totalRecords": 1,
            "totalPages": 1
        },
        "items": [
            {
                "clip": "2562760795",
                "transactionDetails": {
                    "primaryCategoryCode": "A",
                    "deedCategoryCode": "G",
                    "saleDateDerived": "1992-04-13",
                    "saleRecordingDateDerived": "1992-04-13",
                    "saleAmount": 40000,
                    "saleTypeCode": "F",
                    "saleDocumentTypeCode": "ID",
                    "saleDocumentNumber": "8893",
                    "saleBookNumber": "",
                    "salePageNumber": "",
                    "ownershipTransferPercent": null,
                    "multiOrSplitParcelCode": "",
                    "isCashPurchase": false,
                    "isMortgagePurchase": true,
                    "isInterfamilyRelated": true,
                    "isInvestorPurchase": false,
                    "isResale": true,
                    "isShortSale": false,
                    "isForeclosureReo": false,
                    "isForeclosureReoSale": false
                },
                "recordedPropertyAddress": {
                    "streetAddress": "",
                    "streetAddressParsed": {
                        "houseNumber": "31899",
                        "houseNumberSuffix": "",
                        "houseNumber2": "",
                        "direction": "",
                        "streetName": "HIGHWAY 41",
                        "mode": "",
                        "quadrant": "",
                        "unitNumber": ""
                    },
                    "city": "COARSEGOLD",
                    "state": "CA",
                    "zipCode": "936149742",
                    "carrierRoute": "H069",
                    "county": "MADERA"
                },
                "titleCompany": {
                    "name": "",
                    "code": "00006"
                },
                "propertyDetails": {
                    "actualYearBuilt": null,
                    "effectiveYearBuilt": null,
                    "isResidentialProperty": null,
                    "isNewConstruction": false
                },
                "landUseAndZoningCodes": {
                    "propertyTypeCode": "",
                    "landUseCode": "",
                    "stateLandUseDescription": null,
                    "countyLandUseDescription": null,
                    "zoningCode": ""
                },
                "buyerDetails": {
                    "buyerNames": [],
                    "relationshipTypeCode": "",
                    "etalCode": "",
                    "occupancyCode": "",
                    "ownershipRightsCode": "CO",
                    "mailingAddress": {
                        "careOfName": "",
                        "streetAddress": "PO BOX 117",
                        "streetAddressParsed": {
                            "houseNumber": "",
                            "houseNumberSuffix": "",
                            "houseNumber2": "",
                            "direction": "",
                            "streetName": "PO BOX    117",
                            "mode": "",
                            "quadrant": "",
                            "unitNumber": ""
                        },
                        "city": "",
                        "state": "CA",
                        "zipCode": "936140117",
                        "carrierRoute": "B001"
                    },
                    "hasPartialInterest": false,
                    "mailingOptOutIndicator": null
                },
                "sellerDetails": {
                    "sellerNames": [
                        {
                            "sequenceNumber": 1,
                            "fullName": "KERNS TERI"
                        }
                    ]
                }
            }
        ]
    }
}